#### get_matrix

In [2]:
import pandas as pd
import numpy as np


def get_matrix(nb_colors, requests, profile=False):
    requests = requests.split('\n')
    
    matte_wanted = np.zeros((len(requests), nb_colors))
    colors_wanted = np.zeros((len(requests), nb_colors))
    pcount = 0

    for client, line in enumerate(requests):
        pcount += 1
        data = line.split()
        if len(data)%2 != 0:
            raise ValueError('No solution exists')
        
        for i in range(0, len(data), 2):
            pcount += 1
            try:
                color, sheen = int(data[i]), data[i+1]
            except:
                raise ValueError('No solution exists')
            
            #print('Adding {} to matte_wanted[{}][{}]'.format(sheen=='M', client, color))
            matte_wanted[client][color-1] = sheen == 'M'
            #print('Adding 1 to colors_wanted[{}][{}]'.format(client, color))
            colors_wanted[client][color-1] = 1
    
    if profile:
        return matte_wanted, colors_wanted, pcount
    else:
        matte_wanted = pd.DataFrame(matte_wanted, columns=range(nb_colors))
        colors_wanted = pd.DataFrame(colors_wanted, columns=range(nb_colors))
        return matte_wanted, colors_wanted

#### Create matrices

In [ ]:
nb_colors = 5
requests = '''1 M 3 G 5 G
5 M
2 G 3 M 4 G
3 G'''
matte, colors = get_matrix(nb_colors, requests)
matte = pd.DataFrame(matte, columns=range(nb_colors))
colors = pd.DataFrame(colors, columns=range(nb_colors))
matte
colors

#### Create batch & satisfied vectors

In [ ]:
batch = np.zeros(nb_colors)
sheen_chosen = np.zeros(nb_colors)
client_satisfied = np.zeros(len(matte))

### Procedural

#### Find unique requests

##### Reduce the colors matrix by rows

rows in colors with a single value
⇒ That customer only wants one color and we have to respect his sheen preference

In [ ]:
mask_rows = colors.sum(axis=1) == 1
mask_rows

In [ ]:
colors_with_reduced_rows = colors[mask_rows]
colors_with_reduced_rows
mask_cols = colors_with_reduced_rows.sum(axis=0) == 1
mask_cols

##### reduce the colors matrix by cols

In [ ]:
colors_reduced = colors_with_reduced_rows.loc[:, colors_with_reduced_rows.sum(axis=0) == 1]
colors_reduced

##### Update batch and satisfied

In [ ]:
locs = np.where(colors_reduced == 1)
for x, y in locs:
    row = colors_reduced.index[x]     # client id
    col = colors_reduced.columns[y]   # color id
    # Set color sheen as chosen
    sheen_chosen[col] = 1
    # Update color's sheen
    sheen = matte.loc[row, col]
    batch[col] = sheen
    # Set all customers who requested this color/sheen as satisfied
    mask = (colors[col] == 1) & (matte[col] == sheen)
    for row in colors[mask].index:
        client_satisfied[row] = 1

In [ ]:
client_satisfied
sheen_chosen
batch

#### Clear vals from original [wip]

In [ ]:
colors
matte

In [ ]:
mask = colors.iloc[client_satisfied == 0, sheen_chosen == 0] == 1
mask
matte_reduced = matte.iloc[client_satisfied == 0, sheen_chosen == 0]
matte_reduced

In [ ]:
aa = matte_reduced[mask]
aa.notnull()

In [ ]:
matte_reduced[aa.notnull()].min(axis=1)

### Functional

##### 1st: Reduce uniques

In [ ]:
def reduce_uniques(colors, matte, batch, sheen_chosen, client_satisfied):
    #print('in reduce_uniques')
    # First, reduce data matrix to unsatisfied clients and unchosen sheen colors
    colors_reduced = colors.iloc[client_satisfied == 0, sheen_chosen == 0]
    matte_reduced = matte.iloc[client_satisfied == 0, sheen_chosen == 0]
    
    #--------------------------------
    # Then, search for unique requests
    colors_rows = colors_reduced[colors_reduced.sum(axis=1) == 1]
    colors_target = colors_rows.loc[:, colors_rows.sum(axis=0) > 0]
    
    if colors_target.size == 0:
        # No more unique rows to reduce, quit
        return batch, sheen_chosen, client_satisfied
    
    matte_target = matte_reduced.loc[colors_target.index, colors_target.columns]
    matte_sum = matte_target.sum()

    if (matte_sum[matte_sum != 0] != colors_target.sum()[matte_sum != 0]).any():
        # Unique requests have different sheen requirements, impossible to process
        raise ValueError('No solution exists')
    
    # Update batch, sheen_chosen and client_satisfied
    locs = np.where(colors_target == 1)
    #--------------------------------
    
    for x, y in zip(*[l.tolist() for l in locs]):
        row = colors_target.index[x]     # client id
        col = colors_target.columns[y]   # color id
        
        # Set color sheen as chosen
        sheen_chosen[col] = 1
        # Update color's sheen
        sheen = matte.loc[row, col]
        batch[col] = sheen
        # Set all customers who requested this color/sheen as satisfied
        mask = (colors[col] == 1) & (matte[col] == sheen)
        for row in colors[mask].index:
            client_satisfied[row] = 1

    return reduce_uniques(colors, matte, batch, sheen_chosen, client_satisfied)

##### 2nd: Find glossy requests

In [ ]:
def find_glossy(colors, matte, batch, sheen_chosen, client_satisfied):
    #print('in find_glossy')
    # First, reduce data matrix to unsatisfied clients and unchosen sheen colors
    colors_reduced = colors.iloc[client_satisfied == 0, sheen_chosen == 0]
    matte_reduced = matte.iloc[client_satisfied == 0, sheen_chosen == 0]
    
    #-------------------
    # Then, search for glossy requests
    colors_target = matte_reduced[colors_reduced == 1].dropna(axis=1)
        
    if colors_target.size == 0:
        return batch, sheen_chosen, client_satisfied
    
    locs = np.where(colors_target == 0)
    #--------------------
    
    for x, y in zip(*[l.tolist() for l in locs]):
        row = colors_target.index[x]     # client id
        col = colors_target.columns[y]   # color id
        
        # Set color sheen as chosen
        sheen_chosen[col] = 1
        # Update color's sheen
        sheen = matte.loc[row, col]
        batch[col] = sheen
        # Set all customers who requested this color/sheen as satisfied
        mask = (colors[col] == 1) & (matte[col] == sheen)
        for row in colors[mask].index:
            client_satisfied[row] = 1
            
    return find_glossy(colors, matte, batch, sheen_chosen, client_satisfied)

##### 3rd: Find remaining

In [ ]:
def find_remaining(colors, matte, batch, sheen_chosen, client_satisfied):
    #print('in remaining')
    # First, reduce data matrix to unsatisfied clients and unchosen sheen colors
    colors_reduced = colors.iloc[client_satisfied == 0, sheen_chosen == 0]
    matte_reduced = matte.iloc[client_satisfied == 0, sheen_chosen == 0]
    
    #--------------------------------
    colors_target = matte_reduced[colors_reduced == 1]
    
    locs = np.where(colors_target == 1)
    #--------------------------------
    
    for x, y in zip(*[l.tolist() for l in locs]):
        row = colors_target.index[x]     # client id
        col = colors_target.columns[y]   # color id
        
        # Set color sheen as chosen
        sheen_chosen[col] = 1
        # Update color's sheen
        sheen = matte.loc[row, col]
        batch[col] = sheen
        # Set all customers who requested this color/sheen as satisfied
        mask = (colors[col] == 1) & (matte[col] == sheen)
        for row in colors[mask].index:
            client_satisfied[row] = 1
            
    return batch, sheen_chosen, client_satisfied

##### Get batch

In [ ]:
def get_batch(nb_colors, requests):
    # Create matrices
    matte, colors = get_matrix(nb_colors, requests)
    #matte = pd.DataFrame(matte, columns=range(nb_colors))
    #colors = pd.DataFrame(colors, columns=range(nb_colors))
    
    # Create vectors
    batch = np.zeros(nb_colors)
    sheen_chosen = np.zeros(nb_colors)
    client_satisfied = np.zeros(len(matte))

    # Compute solution
    result = reduce_uniques(colors, matte, batch, sheen_chosen, client_satisfied)
    result = find_glossy(colors, matte, *result)
    batch, sheen_chosen, client_satisfied = find_remaining(colors, matte, *result)
    #print('b', batch)
    #print('s', sheen_chosen)
    #print('c', client_satisfied)
    if not client_satisfied.all():
        raise ValueError('No solution exists')
    else:
        return ' '.join(['M' if m else 'G' for m in batch])

### Refactored functional

In [3]:
def locs_for_unique_requests(colors, matte):
    colors_rows = colors[colors.sum(axis=1) == 1]
    colors_target = colors_rows.loc[:, colors_rows.sum(axis=0) > 0]
    if colors_target.size == 0:
        # No more unique rows to reduce, quit
        return False, False

    matte_target = matte.loc[colors_target.index, colors_target.columns]
    matte_sum = matte_target.sum()

    if (matte_sum[matte_sum != 0] != colors_target.sum()[matte_sum != 0]).any():
        # Unique requests have different sheen requirements, impossible to process
        raise ValueError('No solution exists')
    
    # Update batch, sheen_chosen and client_satisfied
    locs = np.where(colors_target == 1)
    return colors_target, locs

def locs_for_glossy(colors, matte):
    colors_target = matte[colors == 1].dropna(axis=1)
    if colors_target.size == 0:
        return False, False
    
    locs = np.where(colors_target == 0)
    return colors_target, locs

def locs_for_remaining(colors, matte):
    colors_target = matte[colors == 1]
    if colors_target.size == 0:
        return False, False
    locs = np.where(colors_target == 1)
    return colors_target, locs


In [22]:
def get_batch2(nb_colors, requests, profile=False):
    pcount = 0
    # Create matrices
    matte, colors = get_matrix(nb_colors, requests)
    #print(matte.shape, colors.shape)
    
    # Create vectors
    batch = np.zeros(nb_colors)
    sheen_chosen = np.zeros(nb_colors)
    client_satisfied = np.zeros(len(matte))
    
    def compute_iter(f, pcount):
        pcount += 1
        # reduce data matrix to unsatisfied clients and unchosen sheen colors
        colors_reduced = colors.iloc[client_satisfied == 0, sheen_chosen == 0]
        matte_reduced = matte.iloc[client_satisfied == 0, sheen_chosen == 0]

        # Find targets and locations
        colors_target, locs = f(colors_reduced, matte_reduced)
        if colors_target is False:
            return
        #print('in', f.__name__)

        for x, y in zip(*[l.tolist() for l in locs]):
            pcount += 1
            row = colors_target.index[x]     # client id
            col = colors_target.columns[y]   # color id

            # Set color sheen as chosen
            sheen_chosen[col] = 1
            # Update color's sheen
            sheen = matte.loc[row, col]
            batch[col] = sheen
            # Set all customers who requested this color/sheen as satisfied
            mask = (colors[col] == 1) & (matte[col] == sheen)
            for row in colors[mask].index:
                pcount += 1
                client_satisfied[row] = 1
        compute_iter(f, pcount)

    # Compute solution
    for f in [locs_for_unique_requests, locs_for_glossy, locs_for_remaining]:
        pcount += 1
        if client_satisfied.all():
            break  # solution already found, quit
        compute_iter(f, pcount)
    
    if not client_satisfied.all():
        raise ValueError('No solution exists')
    else:
        if profile:
            return pcount
        else:
            return ' '.join(['M' if m else 'G' for m in batch])

### Passing Tests

In [23]:
nb_colors = 5
requests = '''1 M 3 G 5 G
5 M
2 G 3 M 4 G'''
get_batch2(nb_colors, requests)

'G G G G M'

In [24]:
nb_colors = 5
requests = '''2 M
5 G
1 G
5 G 1 G 4 M
3 G
5 G
3 G 5 G 1 G
3 G
2 M
5 G 1 G
2 M
5 G
4 M
5 G 4 M'''
get_batch2(nb_colors, requests)

'G M G M G'

In [8]:
nb_colors = 2
requests = '''1 G 2 M
1 M'''
get_batch2(nb_colors, requests)

'M M'

In [9]:
nb_colors = 3
requests = '''1 M 2 M 3 G
1 M 2 G 3 M'''
get_batch2(nb_colors, requests)

'G G G'

In [10]:
nb_colors = 3
requests = '''1 M 2 M 3 G
1 M 3 M
2 M 3 M'''
get_batch2(nb_colors, requests)

'M M G'

### Failing test

##### bad input

In [ ]:
nb_colors = 2
requests = '''1M'''

get_batch2(nb_colors, requests)

##### no solution

In [ ]:
import unittest

nb_colors = 1
requests = '''1 M
1 G'''

get_batch2(nb_colors, requests)

### Complexity

check with timeit

In [11]:
nb_colors = 5
requests = '''2 M
5 G
1 G
5 G 1 G 4 M
3 G
5 G
3 G 5 G 1 G
3 G
2 M
5 G 1 G
2 M
5 G
4 M
5 G 4 M'''
%timeit get_batch2(nb_colors, requests)

10 loops, best of 3: 14.2 ms per loop


Make a data_gen function to randomly output requests

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import random

def data_gen(nb_clients, nb_colors):
    raw = ''
    for _ in range(nb_clients):
        nb_choices = random.choice(range(1, nb_colors))
        colors = sorted({random.choice(range(nb_colors)) for _ in range(nb_choices)})
        request = ['{} {}'.format(i+1, random.choice('GM')) for i in colors]
        raw += ' '.join(request) + '\n'
    return raw


wrap the get_batch function to handle errors

In [ ]:
def get_profiled_batch(nb_colors, requests):
    try:
        return get_batch2(nb_colors, requests, profile=True)
    except:
        return 0

Check complexity per nb of colors (considering 100 clients)

In [ ]:
x = range(2, 10)
y = [get_profiled_batch(i, data_gen(5, i)) for i in x]
plt.plot(x, y)